# FDM Example
เอกสารนี้เตรียมไว้เป็นตัวอย่างสำหรับการทดลองถอดรหัสข้อมูลที่ถูกมัลติเพล็กซ์ไว้ด้วยกลไก Frequency Division Multiplexing (FDM)

In [12]:
from sigproc import Signal,start_notebook
from analog import Qam
bkp = start_notebook()

Loading BokehJS ...

Notebook session ready.


## อ่านอินพุทไฟล์
* อ่านอินพุทไฟล์และลองแสดงค่าสัญญาณในแกนเวลา จะเห็นได้ว่าไม่สามารถตีความข้อมูลภายในสัญญาณด้วยตาเปล่าได้เนื่องจากมีการผสมกันของข้อมูลหลายสตรีมที่กระจายกันอยู่ที่ความถี่ต่าง ๆ
* เมื่อตรวจสอบสัญญาณในโดเมนความถี่ จะเห็นว่ามีสัญญาณกระจายตัวอยู่ 3 ช่องสัญญาณย่อย ที่ความถี่พาหะ 100 Hz, 200 Hz และ 300 Hz แต่ละช่องสัญญาณย่อยมีแบนด์วิธด์ประมาณ 80 Hz

In [17]:
s = Signal(wav_file="example.wav")
s.plot_time()
s.plot_freq(fig_options=dict(x_range=(0,500)))

## คัดกรองช่องสัญญาณที่ต้องการ
* โค้ดด้านล่างคัดลอกสัญญาณขึ้นมาใหม่และกรองออกมาเฉพาะสัญญาณในย่าน 160 - 240 Hz โดยใช้เมท็อด `filter()` ที่มีอยู่ในคลาส `Signal`
* ค่าสัญญาณในโดเมนความถี่แสดงให้เห็นว่าสัญญาณเหลือเพียงความถี่เฉพาะในย่าน 160 - 240 Hz เท่านั้น
* ค่าสัญญาณในโดเมนเวลาเริ่มแยกความแตกต่างของซิมโบลได้ด้วยตาเปล่าได้บ้าง แต่จะเห็นว่าสัญญาณบิดเบี้ยวไปบางส่วนเนื่องจากสัญญาณดั้งเดิมผ่านการกรองด้วย band-pass filter ก่อนเข้ากระบวนการ FDM เพื่อไม่ให้กวนช่องสัญญาณใกล้เคียง อย่างไรก็ตามเราจะถอดสัญญาณ baseband ออกมาจากสัญญาณนี้ก่อนเพื่อให้การตีความง่ายขึ้น

In [19]:
s2 = s.copy().filter((160,240))
s2.plot_freq(fig_options=dict(x_range=(0,500)))
s2.plot_time()

## ถอดสัญญาณ baseband
* สัญญาณ baseband ทั้งส่วนของ I และ Q ทำได้โดยการคูณสัญญาณเข้ากับคลื่นพาหะที่มุม 0 และ 90 องศา
* คลื่นพาหะสร้างได้จากเมท็อด `generate_frequency()` ในคลาส `Qam`
* ผลลัพธ์จากการคูณสัญญาณต้องนำมาผ่าน low-pass filter เพื่อให้ได้สัญญาณ baseband ออกมา

In [23]:
carrier_i = Qam.generate_frequency(200,s1.duration,phase=0)
carrier_q = Qam.generate_frequency(200,s1.duration,phase=90)
baseband_i = (s2*carrier_i*2).filter(200)
baseband_q = (s2*carrier_q*2).filter(200)
print("Baseband I")
baseband_i.plot_time()
print("Baseband Q")
baseband_q.plot_time()

Baseband I


Baseband Q


## ตีความสัญญาณ baseband
สัญญาณ baseband ข้างต้นค่อนข้างชัดเจนพอที่จะตีความได้ด้วยตาเปล่า แต่ถ้าหากต้องการคำนวณค่า amplitude/phase อัตโนมัติสามารถดูตัวอย่างจากโค้ดด้านล่างได้

In [25]:
import numpy as np

# ตรวจสอบค่าสัญญาณที่เวลา 0.1 วินาที
z = baseband_i.time_function(0.1) + baseband_q.time_function(0.1)*1j
print("Amplitude =",np.abs(z))
print("Phase =",np.degrees(np.angle(z)))

Amplitude = 0.290043826955
Phase = -90.3513183056
